In [34]:
# REDO SCRAPE TO MAKE youtube_stats.csv



In [35]:
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd
import matplotlib.pyplot as plt
# import ytcreds
import json
import credentials
import spacy

nlp = spacy.load('en_core_web_sm')

DEVELOPER_KEY = credentials.API_KEY
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

In [36]:
### search Youtube, iterate through pages and get list of responses
###OUT: response_list: list of dictionaries
#use youtube.search().list(part='id, snippet')

query="why I quit my job"
order='relevance'
max_results=1000
location=None
location_radius=None

responses_list =[]

for i in range(0,max_results,50):
    print(i)
    if i ==0:
        next_token=None
        prev_token = None
    search_response = youtube.search().list(
        q=query,
        type="video",
        pageToken=next_token,
        order = order,
        part="id,snippet",
        maxResults=max_results,
        location=location,
        locationRadius=location_radius).execute()
    
    responses_list.append(search_response)
    #GET NEXT PAGE OF RESULTS
    if 'nextPageToken' in search_response.keys():
        next_token=search_response['nextPageToken']
    else:
        next_token=None
        
    if 'prevPageToken' in search_response.keys():
        prev_token=search_response['prevPageToken']
    else:
        prev_token=None

#     print("----")

    
#     print(responses_list[-1])

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [37]:
#response list is a list of dictionaries - 'items' stores individual video info
#responses_list[0]['items'][0]
responses_list[0]

{'kind': 'youtube#searchListResponse',
 'etag': 'GDX_WIRSobCpOOC9MlFYmYbVLCw',
 'nextPageToken': 'CDIQAA',
 'regionCode': 'US',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'uyPLYoDQt1bpxfOTw87PVw3bicU',
   'id': {'kind': 'youtube#video', 'videoId': 'PgfzN1xfgLc'},
   'snippet': {'publishedAt': '2021-09-07T22:30:00Z',
    'channelId': 'UCCFodX6dcHPnkcNkyOkjRLA',
    'title': 'Why I Quit my Job With No Backup Plan | The Perspective That Changed My Life',
    'description': 'Apply for Life & Mindset Coaching: https://www.chriscascella.com/coaching-application/ Sign up for my newsletter Free Thoughts ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/PgfzN1xfgLc/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/PgfzN1xfgLc/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/PgfzN1xfgLc/hqdefault.jpg'

In [38]:

# for response in responses_list:
#     print(response.keys())
#     for key in response.keys():

#         print(response['items'])
#         if key =="items":
#             item_list = response['items']
#             for item in item_list:
#                 video_dict = {}
#     #             print(item)
#                 title=item['snippet']['title']

#                 desc = item['snippet']['description']
#                 video_id = item['id']['videoId']
#                 channel_id = items['snippet']['channelId']
#                 print(desc)

#         print("____")

In [39]:
#create list of ids to iterate through - remove the layer of lists given for individual queries of youtube.search()

video_list = []
for response in responses_list:
#     print(response.keys())
    videos_list = response['items']
    for item in videos_list:
        
        video_title=item['snippet']['title']
#         desc = item['snippet']['description'] #only gives snippet of desc - add when getting transcripts
        video_id = item['id']['videoId']
        channel_id = item['snippet']['channelId']
        published_at = item['snippet']['publishedAt']
        video_dict = {
            "videoId": video_id,
            "videoTitle": video_title,
            "channelId": channel_id,
            'publishedAt': published_at
        }
        
        video_list.append(video_dict)
len(video_list)

        

999

In [40]:
#Create individual lists and store items from video_list dictionaries

title = []
channelId = []
channelTitle = []
categoryId = []
publishedAt=[]
videoId = []
viewCount = []
likeCount = []
dislikeCount = []
commentCount = []
favoriteCount = []
category = []
tags = []
videos = []
descriptions=[]
i = 0
for video in video_list:
    if i % 100 == 0:
        print("VIDEOS PROCESSED: ", i)
    videoId.append(video['videoId'])
    title.append(video['videoTitle'])
    publishedAt.append(video['publishedAt'])
    stats = youtube.videos().list(
        part='statistics, snippet',
        id=video['videoId']).execute()
    channelId.append(stats['items'][0]['snippet']['channelId']) 
    channelTitle.append(stats['items'][0]['snippet']['channelTitle']) 
    categoryId.append(stats['items'][0]['snippet']['categoryId']) 
    favoriteCount.append(stats['items'][0]['statistics']['favoriteCount'])
    try:
        viewCount.append(stats['items'][0]['statistics']['viewCount']) 
    except:
        viewCount.append("Not available") 
    #Not every video has likes/dislikes enabled so they won't appear in JSON response
    try:
        likeCount.append(stats['items'][0]['statistics']['likeCount'])
    except:
   #Good to be aware of Channels that turn off their Likes
#         print("Video titled {0}, on Channel {1} Likes Count is not available".format(stats['items'][0]['snippet']['title'],
#                                                                                      stats['items'][0]['snippet']['channelTitle']))
#         print(stats['items'][0]['statistics'].keys())
    #Appends "Not Available" to keep dictionary values aligned
        likeCount.append("Not available")

    try:
        dislikeCount.append(stats['items'][0]['statistics']['dislikeCount'])     
    except:
        #Good to be aware of Channels that turn off their Likes
#         print("Video titled {0}, on Channel {1} Dislikes Count is not available".format(stats['items'][0]['snippet']['title'],
#                                                                                         stats['items'][0]['snippet']['channelTitle']))
#         print(stats['items'][0]['statistics'].keys())
        dislikeCount.append("Not available")

    if 'commentCount' in stats['items'][0]['statistics'].keys():
        commentCount.append(stats['items'][0]['statistics']['commentCount'])
    else:
        commentCount.append(0)

    if 'tags' in stats['items'][0]['snippet'].keys():
        tags.append(stats['items'][0]['snippet']['tags'])
    else:
        tags.append("No Tags")
        
    if 'description' in stats['items'][0]['snippet'].keys():
        descriptions.append(stats['items'][0]['snippet']['description'])
    else:
        #I'm not a fan of empty fields
        tags.append("No Description")
    
    #a given video is equivelant to stats['items'][0]
    i += 1

VIDEOS PROCESSED:  0
VIDEOS PROCESSED:  100
VIDEOS PROCESSED:  200
VIDEOS PROCESSED:  300
VIDEOS PROCESSED:  400
VIDEOS PROCESSED:  500
VIDEOS PROCESSED:  600
VIDEOS PROCESSED:  700
VIDEOS PROCESSED:  800
VIDEOS PROCESSED:  900


In [41]:
#write JSON to store video information for all
youtube_dict = {'tags':tags,'channelId': channelId,'channelTitle': channelTitle,
                'categoryId':categoryId,'publishedAt': publishedAt,'title':title,'videoId':videoId,
                'viewCount':viewCount,'likeCount':likeCount,'dislikeCount':dislikeCount,
                'commentCount':commentCount,'favoriteCount':favoriteCount,
               "description": descriptions}

# for key in youtube_dict.keys():
#     print(key)
#     print("LENGTH: ", len(youtube_dict[key]))
#     print(youtube_dict[key][:5])
    
#     print("---")
youtube_dict.items()

dict_items([('tags', ['No Tags', ['i quit my job', 'quit job', 'i quit', 'adulting series', 'iris wellen', 'moya mawhinney', "the scariest thing i've ever done", 'making life decisions', 'big life update', 'life update', 'adulting', 'how to quit your job', 'quitting my job', 'new phase in my life', 'new chapter', 'doing youtube full time', 'i quit my job to do youtube', 'i quit my job for youtube', 'fulltime social media', 'full time youtube', 'i quit my 9 til 5', 'i quit my fulltime job', 'quiet quitting', 'self employed'], ['Quit my job', 'Great Resignation', 'Self Employed', 'Done', "I'm Done"], ['i quit my job', 'i quit my corporate job', 'quit your job', 'quiet quitting', 'quit my job', 'quit your job motivation', 'should i quit my job', 'mental health', 'why i quit my job', 'should you quit your job', 'signs you should quit your job', 'great resignation', 'corporate 9-5', 'corporate 9 to 5', 'corporate job life', 'quitting my job', 'taking risks', 'taking risks motivation', 'ment

In [42]:
from youtube_transcript_api import YouTubeTranscriptApi

In [43]:
test_id = "PgfzN1xfgLc"
YouTubeTranscriptApi.get_transcript(test_id,languages=['en'])

for i, video_id in enumerate(youtube_dict['videoId']):
     print(video_id)

PgfzN1xfgLc
NhVg9qdHWiY
6ecjnRHt5l8
qfJlP8VhYDg
84FPm7oAQG8
SZEo1KFjTn4
Cv1w7r1Egik
w0-B3JloSfE
VCXp7_4NUnA
ZvCGppdE2E0
E1YRzxGKyEA
3e_ARQMXreM
l4bZ3wuOQPY
ILxBLYfZENg
Hr8UjzeGm-M
dFKM-dojNf8
zBQa48lLuVg
_MQlj6lcxV8
BwMsYcxi2u4
C9foNXwifOA
MKHqJD5syqk
iVLiRWD3gAM
6HKLmRhKiTc
ppzdevWY62Q
7Zvctvsk9Kw
wDqhgTUaQD0
A6ZwbWKlAS8
B4S2pQ4EzPA
RmG3qdWuUZY
Mdq1_1-8xzw
zZY8zZHEcAQ
rtIbI_qra84
rcjW24_El-0
SJwGsZBx34g
xxPdTjk7beo
DMAXo1_8HI0
S_D4yJavp8M
qL-vUA3KIa4
Inaw4g3mBw4
4IoAH-dXA8I
9sdwaf76D_Y
kV8TYwLbhxs
PGVTH3rcpLE
6Kh06oTXlEw
X4xqrUct7_E
1k6CAQomyTU
BIMIkwu2vCU
UMJR76Wlrp0
cYP9WJSJSfY
rg0h0UOqUUI
r3vrmX8f1q0
UDUp3ytihvY
1k6CAQomyTU
N77Ys4Dr2oo
RDc_Pl1ERUI
CPwsufhMMmc
rcjW24_El-0
2i-AgB_HJaI
nf6fBS_wmEY
nX-UyqlQB_U
stLEmSa3DrQ
_GxTGs-va-w
UMJR76Wlrp0
wvtcFOsc3-U
-eSKRuwfpvk
rg0h0UOqUUI
cYP9WJSJSfY
cZEehdw_PDc
eNEC0m6o6vM
fsxBZKn8AsE
Kdc-e2nOtNw
8TcF-zyv_9c
NIIlYWXM-w8
4un746q4NVw
Ju07vEFGRbk
xt4DUXWUH4U
6XhcZNEQ5Yk
Nt2-mSA1rBU
XXyk_PT9LiM
Bv-wh_PBbtc
dlo0AFcl-nM
uy0stp6DFp8
L4dGMkJCxpI
TTqp

In [44]:
## EXP 
transcripts_sents = []
transcripts_strings = []
transcripts_parsed = []
for i, video_id in enumerate(youtube_dict['videoId']):
    print(video_id)
    transcript_sentlist = []
    try:
        eng_transcript = YouTubeTranscriptApi.get_transcript(video_id,languages=['en'])
        print(eng_transcript[0:5])
        transcript_sentlist = [str(x['text']).replace("\xa0", "") for x in eng_transcript]
        transcript_joined = " ".join(transcript_sentlist)
        #transcript_sentlist is broken up by timestamp - use Spacy to break up using dependancy parse
#         document = nlp(transcript_joined)
#         transcript_parsed = [x for x in document.sents]
    except:
        print("NONE")
        transtript_sentlist = None
        transcript_joined = None
#         transcript_parsed = None
#         pass
    transcripts_sents.append(transcript_sentlist)
    transcripts_strings.append(transcript_joined)
#     transcripts_parsed.append(transcript_parsed)
    if i % 25 ==0:
        print(f"PROCESSED: {i}")
#         break
print(f"COMPLETED")
youtube_dict['transcripts_raw'] = transcripts_sents
youtube_dict['transcript_strings'] = transcripts_strings
# youtube_dict['transcript_parsed'] = transcripts_parsed

PgfzN1xfgLc
[{'text': "what's up everyone welcome to the video", 'start': 0.0, 'duration': 3.84}, {'text': "today i'm going to be talking about why", 'start': 2.0, 'duration': 3.279}, {'text': 'i decided to quit my job without a', 'start': 3.84, 'duration': 2.959}, {'text': 'backup plan i spent a lot of time', 'start': 5.279, 'duration': 3.121}, {'text': 'wrestling with this decision before i', 'start': 6.799, 'duration': 3.121}]
PROCESSED: 0
NhVg9qdHWiY
[{'text': '[Music]', 'start': 1.19, 'duration': 11.179}, {'text': 'thank you', 'start': 12.74, 'duration': 3.059}, {'text': 'foreign', 'start': 29.64, 'duration': 22.669}, {'text': '[Music]', 'start': 31.73, 'duration': 20.579}, {'text': "I don't even know where to start oh my", 'start': 55.739, 'duration': 5.221}]
6ecjnRHt5l8
[{'text': "ladies and gentlemen damusicaviro's 100", 'start': 0.0, 'duration': 6.839}, {'text': 'videos ago I called my mom and I said', 'start': 3.06, 'duration': 8.82}, {'text': "Mama I'mma Be a Star and she sa

[{'text': 'This is one example of quiet\nquitting, a trend that has been', 'start': 12.03, 'duration': 3.39}, {'text': 'dominating social media and\nespecially Tik Tok. In July,', 'start': 15.42, 'duration': 3.57}, {'text': 'Zaid Khan, a 24-year-old\nengineer from New York, posted a', 'start': 18.99, 'duration': 3.36}, {'text': 'video about quiet quitting, and\nit went viral. Since then the', 'start': 22.35, 'duration': 3.51}, {'text': 'trend has spread like wildfire\nwith hashtags like', 'start': 25.86, 'duration': 2.61}]
6HKLmRhKiTc
[{'text': "every time I think I'm the only one", 'start': 0.0, 'duration': 2.94}, {'text': 'feeling a certain way I remind myself', 'start': 1.5, 'duration': 3.42}, {'text': "that there's seven billion people in", 'start': 2.94, 'duration': 3.72}, {'text': "this world I'm not the only one having", 'start': 4.92, 'duration': 3.66}, {'text': 'this feeling someone else is having this', 'start': 6.66, 'duration': 2.64}]
ppzdevWY62Q
[{'text': 'quitting your jo

[{'text': '[Music]', 'start': 0.0, 'duration': 6.08}, {'text': 'being a tech reviewer on youtube is', 'start': 3.84, 'duration': 4.24}, {'text': 'kind of a funny job i get emails asking', 'start': 6.08, 'duration': 3.28}, {'text': 'me to review everything from foot', 'start': 8.08, 'duration': 3.36}, {'text': 'massages to smart handcuffs to', 'start': 9.36, 'duration': 3.92}]
kV8TYwLbhxs
[{'text': 'Oh, my God.', 'start': 1.56, 'duration': 0.376}, {'text': "There's something inside of me.", 'start': 2.002, 'duration': 2.711}, {'text': "Hey yall, it's Chimdi", 'start': 4.713, 'duration': 1.793}, {'text': 'So I have quit my job.', 'start': 6.506, 'duration': 1.835}, {'text': 'As you can see\nfrom the title of this video,', 'start': 8.341, 'duration': 2.795}]
PGVTH3rcpLE
[{'text': 'hey guys so i am about to embark on an', 'start': 0.32, 'duration': 4.72}, {'text': 'adventure here at fifth water hot', 'start': 2.8, 'duration': 4.64}, {'text': 'springs i know so many of you like to', 'start'

[{'text': 'foreign', 'start': 0.0, 'duration': 2.42}, {'text': 'this is a story of how I quit my', 'start': 6.299, 'duration': 5.461}, {'text': 'six-figure job in sales at 24 to take a', 'start': 8.76, 'duration': 4.2}, {'text': 'chance on myself', 'start': 11.76, 'duration': 4.619}, {'text': 'so at the time I was making 120 000 and', 'start': 12.96, 'duration': 4.8}]
wvtcFOsc3-U
NONE
-eSKRuwfpvk
[{'text': 'maybe earlier than that', 'start': 13.04, 'duration': 4.88}, {'text': 'and i tried my absolute best to let', 'start': 14.88, 'duration': 5.36}, {'text': 'everyone here know that i am a valued', 'start': 17.92, 'duration': 3.92}, {'text': 'associate', 'start': 20.24, 'duration': 5.76}, {'text': 'despite that you a team lead', 'start': 21.84, 'duration': 6.8}]
rg0h0UOqUUI
[{'text': "good yeah all right hey guys I'm hpvp", 'start': 0.96, 'duration': 6.66}, {'text': 'here back with another MC PVP Hunger', 'start': 4.92, 'duration': 3.38}, {'text': 'Games', 'start': 7.62, 'duration': 5.2

[{'text': 'uh', 'start': 25.359, 'duration': 3.0}, {'text': 'today is my last working day 22nd april', 'start': 37.52, 'duration': 7.0}, {'text': 'friday 2022.', 'start': 40.8, 'duration': 3.72}, {'text': 'content related to that i know that you', 'start': 46.16, 'duration': 4.239}, {'text': 'know i constantly place on my work', 'start': 48.559, 'duration': 3.68}]
aisiHrAWUaQ
[{'text': "hey I'm Emma I quit my job as a coffee", 'start': 0.0, 'duration': 4.56}, {'text': 'shop manager to become a full-time', 'start': 2.76, 'duration': 3.66}, {'text': 'artist why did I do this', 'start': 4.56, 'duration': 3.959}, {'text': 'because being an artist has been my', 'start': 6.42, 'duration': 4.02}, {'text': 'dream since I was a little girl and up', 'start': 8.519, 'duration': 4.261}]
5tSZ8nWbU4A
[{'text': 'I ran the raw last night okay sounds', 'start': 0.0, 'duration': 3.72}, {'text': "like I'm done here he said that", 'start': 2.22, 'duration': 2.94}, {'text': 'basically probably that coach f

...

[{'text': "oh it's so loud outside okay", 'start': 0.06, 'duration': 6.029}, {'text': "testing what's up everyone its Natalie", 'start': 3.419, 'duration': 6.451}, {'text': 'here and today I am filming they must', 'start': 6.089, 'duration': 6.331}, {'text': 'like nerve-wracking video I have had to', 'start': 9.87, 'duration': 4.769}, {'text': 'film in quite a while I feel like I', 'start': 12.42, 'duration': 4.23}]
DEVvdAahkQQ
[{'text': 'Emirates is like that toxic relationship', 'start': 0.0, 'duration': 4.68}, {'text': 'that just keeps pulling you back in I', 'start': 2.46, 'duration': 3.899}, {'text': 'felt like I was in a toxic relationship', 'start': 4.68, 'duration': 4.32}, {'text': 'with my job hey guys welcome to my', 'start': 6.359, 'duration': 4.74}, {'text': "channel my name is Jess so in today's", 'start': 9.0, 'duration': 3.84}]
uKo9VejKpg8
[{'text': "hi guys i'm chelsea i am a full-time", 'start': 0.0, 'duration': 7.71}, {'text': "artist i'm also the voice behind the", '

[{'text': 'so i quit my job', 'start': 4.08, 'duration': 4.24}, {'text': "i've handed him my notice and in four", 'start': 6.0, 'duration': 4.559}, {'text': "weeks time i'm gonna be", 'start': 8.32, 'duration': 4.72}, {'text': 'a full-time youtuber i am incredibly', 'start': 10.559, 'duration': 5.361}, {'text': "excited i'm also really scared as well", 'start': 13.04, 'duration': 4.72}]
K2T-WafRTSQ
[{'text': "so can you quit your job when you're on", 'start': 0.12, 'duration': 5.34}, {'text': 'workers comp you got injured at work', 'start': 3.36, 'duration': 4.38}, {'text': "you're treating you're not yet permanent", 'start': 5.46, 'duration': 3.84}, {'text': "and stationary let's say you're still", 'start': 7.74, 'duration': 4.319}, {'text': "getting treatment haven't seen a qme but", 'start': 9.3, 'duration': 4.919}]
PROCESSED: 925
XSD-sSnTl-k
[{'text': "hey y'all welcome back to my channel in", 'start': 0.03, 'duration': 4.019}, {'text': "today's video I'm gonna be telling you", 'st

[{'text': "hey guys how's it going happy new year i", 'start': 1.439, 'duration': 5.44}, {'text': 'am a month late but better late than', 'start': 4.4, 'duration': 3.359}, {'text': 'never', 'start': 6.879, 'duration': 2.481}, {'text': 'i hope that everyone had a really', 'start': 7.759, 'duration': 3.92}, {'text': 'awesome holiday season i hope that', 'start': 9.36, 'duration': 6.399}]
QXSquAP2AGA
[{'text': 'i recently quit my job as a management', 'start': 0.56, 'duration': 3.52}, {'text': 'consultant in toronto after working for', 'start': 2.24, 'duration': 3.519}, {'text': "less than a year and today i'm going to", 'start': 4.08, 'duration': 5.4}, {'text': 'tell you why i quite', 'start': 5.759, 'duration': 3.721}, {'text': '[Music]', 'start': 10.38, 'duration': 9.979}]
uMQXysjEkMo
[{'text': 'hello everyone namadhan guri and hello', 'start': 5.359, 'duration': 5.121}, {'text': 'to all the subscribed mgs 2019 17', 'start': 7.2, 'duration': 5.599}, {'text': 'january nikkei now in the 

[{'text': '[Music]', 'start': 0.2, 'duration': 8.609}, {'text': 'hey', 'start': 12.24, 'duration': 3.519}, {'text': "today i'm going to be going over why i", 'start': 13.28, 'duration': 3.68}, {'text': 'quit my job', 'start': 15.759, 'duration': 3.761}, {'text': 'at costco and what do i really think', 'start': 16.96, 'duration': 4.72}]
B9g-eoyoLWA
[{'text': '[Music]', 'start': 0.43, 'duration': 2.34}, {'text': "hey loves I'm Anita welcome back to my", 'start': 6.259, 'duration': 5.081}, {'text': 'channel if this is your first time here', 'start': 9.599, 'duration': 3.601}, {'text': 'do well to click on that subscribe', 'start': 11.34, 'duration': 4.379}, {'text': 'button to join this beautiful family and', 'start': 13.2, 'duration': 4.499}]
imkLwoUrGe4
[{'text': 'and so a lot of that work uh the most', 'start': 0.08, 'duration': 4.88}, {'text': 'kind of crucial work that actually', 'start': 2.48, 'duration': 4.319}, {'text': 'helps move the company forward is not', 'start': 4.96, 'dura

[{'text': "hi everybody it's jessica notes with our", 'start': 0.16, 'duration': 4.48}, {'text': 'bipolar', 'start': 3.439, 'duration': 3.36}, {'text': "and i'm here with another vlog even", 'start': 4.64, 'duration': 4.56}, {'text': 'though i was supposed to be doing a blog', 'start': 6.799, 'duration': 3.521}, {'text': 'today', 'start': 9.2, 'duration': 4.0}]
H3Y7N8h-lbY
[{'text': "i've had this amazing career", 'start': 0.16, 'duration': 6.639}, {'text': "and it's been fun until now", 'start': 3.52, 'duration': 5.2}, {'text': "it's time for me to retire i'm not", 'start': 6.799, 'duration': 3.361}, {'text': 'appreciated anymore', 'start': 8.72, 'duration': 5.52}, {'text': "but i'm scared of the future", 'start': 10.16, 'duration': 6.8}]
uXoDe6HPkYU
[{'text': "it's been almost one year since I left", 'start': 0.0, 'duration': 4.5}, {'text': 'my job to start doing YouTube full time', 'start': 2.46, 'duration': 4.5}, {'text': 'thankfully my channel is still alive', 'start': 4.5, 'durat

In [45]:
# youtube_dict

In [52]:
# new_descs = [str(x) for x in youtube_dict['description']]
# print(new_descs)
# youtube_dict['description'] = new_descs

# trans_parsed = []
# for transcript_parsed in youtube_dict['transcript_parsed']:
#     print(transcript_parsed)
#     if transcript_parsed:
#         new_parsed = [str(x) for x in transcript_parsed]
#     else:
#         new_parsed = None
#     trans_parsed.append(new_parsed)
# youtube_dict['transcript_parsed'] = trans_parsed
# print(len(trans_parsed))

In [53]:
for key, item in youtube_dict.items():
    print(f"KEY AND LENGTH OF ITEMS: {key}, {len(item)}")

KEY AND LENGTH OF ITEMS: tags, 999
KEY AND LENGTH OF ITEMS: channelId, 999
KEY AND LENGTH OF ITEMS: channelTitle, 999
KEY AND LENGTH OF ITEMS: categoryId, 999
KEY AND LENGTH OF ITEMS: publishedAt, 999
KEY AND LENGTH OF ITEMS: title, 999
KEY AND LENGTH OF ITEMS: videoId, 999
KEY AND LENGTH OF ITEMS: viewCount, 999
KEY AND LENGTH OF ITEMS: likeCount, 999
KEY AND LENGTH OF ITEMS: dislikeCount, 999
KEY AND LENGTH OF ITEMS: commentCount, 999
KEY AND LENGTH OF ITEMS: favoriteCount, 999
KEY AND LENGTH OF ITEMS: description, 999
KEY AND LENGTH OF ITEMS: transcripts_raw, 999
KEY AND LENGTH OF ITEMS: transcript_strings, 999


In [54]:
for key in youtube_dict.keys():
    dict_item = youtube_dict[key][0]
    print(key)
    print(dict_item)
    print(type(dict_item))
    print("----")

tags
No Tags
<class 'str'>
----
channelId
UCCFodX6dcHPnkcNkyOkjRLA
<class 'str'>
----
channelTitle
Chris Cascella
<class 'str'>
----
categoryId
22
<class 'str'>
----
publishedAt
2021-09-07T22:30:00Z
<class 'str'>
----
title
Why I Quit my Job With No Backup Plan | The Perspective That Changed My Life
<class 'str'>
----
videoId
PgfzN1xfgLc
<class 'str'>
----
viewCount
293930
<class 'str'>
----
likeCount
14566
<class 'str'>
----
dislikeCount
Not available
<class 'str'>
----
commentCount
1507
<class 'str'>
----
favoriteCount
0
<class 'str'>
----
description
Apply for Life & Mindset Coaching: https://www.chriscascella.com/coaching-application/

Sign up for my newsletter Free Thoughts Thursday: https://www.chriscascella.com/

Follow me on Instagram: https://www.instagram.com/chriscascella/

---------------------------------------------------------------------------------------------------------------------------------------------------------------

A lot of talk about The Great Resignation..

In [55]:
# ###ADD IN 02_PROCESS TRANSCRIPTS CODE
# all_transcripts = []

# id_list = youtube_dict['videoId']

# for i, video_id in enumerate(youtube_dict['videoId']):
    
#     try:
#         transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
#     except:
#         transtript_list = None
#         pass
    
#     all_transcripts.append(transcript_list)
#     if not transcript_list:
#         print("MISSING TRANSCRIPT:", youtube_dict["title"][i])
    
#     if i % 100 == 0:
#         print("PROCESSED: ",i)
        
# youtube_dict['transcript_api'] = all_transcripts

In [56]:
filename="youtube_stats_and_transcripts.json"
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(youtube_dict, f, ensure_ascii=False, indent=4)


In [57]:
type(youtube_dict)

dict

In [13]:
# #REDO WITH ID AND TRANSCRIPT API IN ONE PLACE

# transcript_dict_all = {}
# transcript_dict_man = {}

# for i, video_id in enumerate(youtube_dict['videoId']):
    
#     transcript_api = youtube_dict['transcript_api'][i]
#     try:
#         en_transcript = transcript_api.find_transcript(['en']).fetch()
#         transcript_dict_all[video_id] = en_transcript
#     except:
#         pass
    
# #     #try to find a manual transcript if it exists in transcript_list - not often
# #     try:
# #         en_transcript_manual = transcript_api.find_manually_created_transcript(['en']).fetch()
# #         transcript_dict_man[video_id] = en_transcript_manual
# #     except:
# #         pass
    
#     if i % 200 == 0:
#         print("PROCESSED: ", i)
#         print("---")

PROCESSED:  0
---


In [19]:
len(transcript_dict_all.keys())

96

In [ ]:
# def process_transcripts(raw_dict_in, outfile='processed_transcripts.json'):
    
#     """Writes a json file from the raw transcript dict returned from scrape_transcripts()
    
#     Returns processed_transcripts with video_ids as keys and the following items:
#         list: list of sentence strings from 'text' field of raw_dict
#         str: combined string from 'text' field of raw_dict
    
#     """

#     processed_transcripts = {}
#     for video_id, transcript in raw_dict_in.items():
#         processed_transcripts[video_id] = {
#             "list": [],
#             "str": ""
#         }
#         transcript_list = []
#         transcript_str = ""
#         for item in transcript:
#             text_line = item['text']
#             transcript_list.append(text_line)
#             transcript_str += text_line 
#             transcript_str += " "
#         processed_transcripts[video_id]['list'] = transcript_list
#         processed_transcripts[video_id]['str'] = transcript_str
        
#         return processed_transcripts


# #     with open(outfile, 'w', encoding='utf-8') as f:
# #         json.dump(processed_transcripts, f, ensure_ascii=False, indent=4)

In [21]:
# # process_transcripts(transcript_dict_man, "processed_MAN.json")
# process_transcripts(transcript_dict_all, "processed_ALL.json")

In [22]:
#use youtube.videos().list(part='statistics, snippet')